In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import pandas as pd

## Megadepth-1500

In [ ]:
results = 'megadepth1500/results/results.json'
if not os.path.exists(results):
    raise FileNotFoundError(f"Results file not found: {results}. First run the benchmark!")

with open(results) as f:
    data = json.load(f)

df = pd.DataFrame(data).T.round(3)
df['inlier'] = round(df['inlier']).astype(int)
df['auc_5'] *= 100
df['auc_10'] *= 100
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['budget'] = df.index.str.split(' ').str[2]
df['timestamp'] = df.index.str.split(' ').str[-1]

df = df[['model',  'budget', 'inlier', 'auc_5', 'auc_10', 'timestamp']]
df.sort_values(by=['budget', 'model'], inplace=True, ascending=[True, True])
df.index = range(1, 1+len(df.index))
df

## Graz High Resolution

In [6]:
scale_factors = {'scale_1.0': '4K', 'scale_1.5': 'QHD', 'scale_2.0': 'FHD'}

results = 'graz_high_res/results/results.json'
if not os.path.exists(results):
    raise FileNotFoundError(f"Results file not found: {results}. First run the benchmark!")

with open(results) as f:
    data = json.load(f)

df = pd.DataFrame(data).T.round(3)
df['inlier'] = round(df['inlier']).astype(int)
df['auc_5'] *= 100
df['auc_10'] *= 100
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['resolution'] = [scale_factors[s] for s in df.index.str.split(' ').str[2]]
df['budget'] = df.index.str.split(' ').str[3]
df['timestamp'] = df.index.str.split(' ').str[-1]

df_auc = df[['model', 'budget', 'resolution', 'inlier', 'auc_5', 'auc_10', 'timestamp']].copy()
df_auc.sort_values(by=['resolution', 'model'], inplace=True, ascending=[False, True])
df_auc.index = range(1, 1+len(df_auc.index))
df_auc

,model,budget,resolution,inlier,auc_5,auc_10,timestamp
1,aliked,2048,QHD,154,51.9,65.0,20250903_123543
2,aliked,2048,FHD,198,54.7,67.5,20250902_113656
3,aliked+SANDesc,2048,FHD,194,64.4,75.8,20250902_120240
4,dedode,2048,FHD,161,50.6,64.5,20250902_161105
5,dedode+SANDesc,2048,FHD,166,57.4,70.7,20250902_171857
6,dedode-G,2048,FHD,183,56.3,70.0,20250902_193136
7,disk,2048,FHD,160,40.2,52.8,20250902_123607
8,disk+SANDesc,2048,FHD,169,55.3,68.9,20250902_130314
9,ripe,2048,FHD,148,54.8,67.2,20250902_144131
10,ripe+SANDesc,2048,FHD,147,63.3,75.0,20250902_152144


### Scenes breakdown

In [ ]:
from benchmark_utils import *
keys = list(data.keys())
auc_th = 5
scale = 2 # 1, 1.5, 2

breakdown, total = {}, {}
for key in keys:
    parts = key.split(' ')
    key = f'{parts[0]}_stats_scale_{float(scale)}_{parts[4]}_{parts[6]}'
    model = parts[0]

    df = pd.read_csv(f'graz_high_res/stats/{key}.csv')
    scenes = df.scene_name.unique()
    total[model] = round(pose_auc(df.e_pose, [auc_th])[0] * 100,1)

    for scene in scenes:
        scene_df = df[df.scene_name == scene]
        if scene not in breakdown:
            breakdown[scene] = {}
        breakdown[scene][model] = round(pose_auc(scene_df.e_pose, [auc_th])[0] * 100,1)

breakdown['total'] = total

In [ ]:
df_breakdown = pd.DataFrame.from_dict(breakdown, orient='index').T
df_breakdown